# Kernels

In some cases the problem is that we are looking for a linear boundary when the boundary in our space is non-linear. However is does not mean that there is no space in which this problem have no linear boundary. If we find the mapping to such space the problem could be solved. 

The question is if we can map the features space into more dimensions in such a way to make the problem linearly separable. Is is quite obvious that if we will use the function:
\begin{equation}
K(x) = (x_1^2, \sqrt(2) x_1 x_2, x_2^2)
\end{equation}
then we transfer our problem into $R^3$ space in which it is linearly separable.

Load the data set in the first place

In [3]:
from sklearn.datasets import load_iris
import numpy as np
from sklearn.model_selection import train_test_split
import cvxopt

iris = load_iris()
data_set = iris.data
labels = iris.target

data_set = data_set[labels!=2]
labels = labels[labels!=2]

train_data_set, test_data_set, train_labels, test_labels = train_test_split(
    data_set, labels, test_size=0.2, random_state=15)

train_labels[train_labels<1] = -1
test_labels[test_labels<1] = -1

objects_count = len(train_labels)

The ``build_kernel`` function for building more than just the linear kernel needs to be modifier. The RBF kernel can be written as:
\begin{equation}
K=\exp(-\frac{|x-x'|^{2}}{2\sigma^{2}})
\end{equation}


In [4]:
def build_kernel(data_set, kernel_type='linear'):
    kernel = np.dot(data_set, data_set.T)
    if kernel_type == 'rbf':
        sigma = 1.0
        objects_count = len(data_set)
        b = np.ones((len(data_set), 1))
        kernel -= 0.5 * (np.dot((np.diag(kernel)*np.ones((1, objects_count))).T, b.T)
                         + np.dot(b, (np.diag(kernel) * np.ones((1, objects_count))).T.T))
        kernel = np.exp(kernel / (2. * sigma ** 2))
    return kernel

The training part is the same as in the case of linear kernel:

In [5]:
def train(train_data_set, train_labels, kernel_type='linear', C=10, threshold=1e-5):
    kernel = build_kernel(train_data_set, kernel_type=kernel_type)

    P = train_labels * train_labels.transpose() * kernel
    q = -np.ones((objects_count, 1))
    G = np.concatenate((np.eye(objects_count), -np.eye(objects_count)))
    h = np.concatenate((C * np.ones((objects_count, 1)), np.zeros((objects_count, 1))))

    A = train_labels.reshape(1, objects_count)
    A = A.astype(float)
    b = 0.0

    sol = cvxopt.solvers.qp(cvxopt.matrix(P), cvxopt.matrix(q), cvxopt.matrix(G), cvxopt.matrix(h), cvxopt.matrix(A), cvxopt.matrix(b))

    lambdas = np.array(sol['x'])

    support_vectors_id = np.where(lambdas > threshold)[0]
    vector_number = len(support_vectors_id)
    support_vectors = train_data_set[support_vectors_id, :]

    lambdas = lambdas[support_vectors_id]
    targets = train_labels[support_vectors_id]

    b = np.sum(targets)
    for n in range(vector_number):
        b -= np.sum(lambdas * targets * np.reshape(kernel[support_vectors_id[n], support_vectors_id], (vector_number, 1)))
    b /= len(lambdas)

    return lambdas, support_vectors, support_vectors_id, b, targets, vector_number

The prediction part is a bit different as we need to take RBF kernel into consideration:

\begin{equation}
K=\exp(-\frac{|x-x'|^{2}}{2\sigma^{2}})
\end{equation}

In [6]:
def classify_rbf(test_data_set, train_data_set, lambdas, targets, b, vector_number, support_vectors, support_vectors_id):
    kernel = np.dot(test_data_set, support_vectors.T)
    sigma = 1.0
    c = (1. / sigma * np.sum(test_data_set ** 2, axis=1) * np.ones((1, np.shape(test_data_set)[0]))).T
    c = np.dot(c, np.ones((1, np.shape(kernel)[1])))
    sv = (np.diag(np.dot(train_data_set, train_data_set.T))*np.ones((1,len(train_data_set)))).T[support_vectors_id]
    aa = np.dot(sv,np.ones((1,np.shape(kernel)[0]))).T
    kernel = kernel - 0.5 * c - 0.5 * aa
    kernel = np.exp(kernel / (2. * sigma ** 2))

    y = np.zeros((np.shape(test_data_set)[0], 1))
    for j in range(np.shape(test_data_set)[0]):
        for i in range(vector_number):
            y[j] += lambdas[i] * targets[i] * kernel[j, i]
        y[j] += b
    return np.sign(y)

In [82]:
kernel = np.dot(test_data_set, support_vectors.T)
kernel

array([[37.32, 65.76, 37.5 , 70.65, 65.85, 50.25, 51.33],
       [35.6 , 50.25, 34.55, 56.09, 49.4 , 39.25, 48.78],
       [32.  , 54.23, 32.16, 58.89, 54.31, 41.85, 43.84],
       [42.22, 72.3 , 42.46, 78.29, 72.39, 55.65, 57.91],
       [35.11, 50.29, 33.83, 55.66, 49.34, 38.9 , 48.27],
       [39.6 , 68.98, 39.97, 74.53, 69.18, 53.05, 54.31],
       [32.13, 54.5 , 32.38, 59.27, 54.64, 42.15, 43.97],
       [38.28, 53.14, 36.92, 59.32, 52.04, 41.4 , 52.54],
       [32.62, 47.24, 31.56, 52.28, 46.46, 36.6 , 44.8 ],
       [34.89, 60.56, 35.1 , 65.33, 60.62, 46.45, 47.93],
       [39.14, 67.27, 39.36, 72.74, 67.32, 51.7 , 53.73],
       [29.37, 42.16, 28.45, 46.79, 41.45, 32.75, 40.31],
       [34.72, 49.85, 33.53, 55.15, 48.88, 38.55, 47.76],
       [36.41, 64.05, 36.81, 69.18, 64.38, 49.3 , 49.87],
       [29.18, 42.32, 28.35, 46.94, 41.69, 32.9 , 40.02],
       [32.28, 47.14, 31.54, 52.4 , 46.58, 36.8 , 44.18],
       [40.24, 70.35, 40.65, 76.03, 70.64, 54.15, 55.14],
       [32.03,

In [83]:
c = (1. / 1.0 * np.sum(test_data_set ** 2, axis=1) * np.ones((1, np.shape(test_data_set)[0]))).T
c = np.dot(c, np.ones((1, np.shape(kernel)[1])))
c

array([[60.66, 60.66, 60.66, 60.66, 60.66, 60.66, 60.66],
       [44.23, 44.23, 44.23, 44.23, 44.23, 44.23, 44.23],
       [41.66, 41.66, 41.66, 41.66, 41.66, 41.66, 41.66],
       [73.7 , 73.7 , 73.7 , 73.7 , 73.7 , 73.7 , 73.7 ],
       [43.05, 43.05, 43.05, 43.05, 43.05, 43.05, 43.05],
       [66.91, 66.91, 66.91, 66.91, 66.91, 66.91, 66.91],
       [42.18, 42.18, 42.18, 42.18, 42.18, 42.18, 42.18],
       [51.12, 51.12, 51.12, 51.12, 51.12, 51.12, 51.12],
       [37.2 , 37.2 , 37.2 , 37.2 , 37.2 , 37.2 , 37.2 ],
       [51.5 , 51.5 , 51.5 , 51.5 , 51.5 , 51.5 , 51.5 ],
       [63.7 , 63.7 , 63.7 , 63.7 , 63.7 , 63.7 , 63.7 ],
       [30.09, 30.09, 30.09, 30.09, 30.09, 30.09, 30.09],
       [42.11, 42.11, 42.11, 42.11, 42.11, 42.11, 42.11],
       [57.81, 57.81, 57.81, 57.81, 57.81, 57.81, 57.81],
       [29.77, 29.77, 29.77, 29.77, 29.77, 29.77, 29.77],
       [36.6 , 36.6 , 36.6 , 36.6 , 36.6 , 36.6 , 36.6 ],
       [69.67, 69.67, 69.67, 69.67, 69.67, 69.67, 69.67],
       [35.88,

In [81]:
sv = (np.diag(np.dot(train_data_set, train_data_set.T))*np.ones((1,len(train_data_set)))).T[support_vectors_id]
aa = np.dot(sv,np.ones((1,np.shape(kernel)[0]))).T
aa

array([[28.71, 71.33, 27.32, 83.29, 71.86, 42.25, 54.26],
       [28.71, 71.33, 27.32, 83.29, 71.86, 42.25, 54.26],
       [28.71, 71.33, 27.32, 83.29, 71.86, 42.25, 54.26],
       [28.71, 71.33, 27.32, 83.29, 71.86, 42.25, 54.26],
       [28.71, 71.33, 27.32, 83.29, 71.86, 42.25, 54.26],
       [28.71, 71.33, 27.32, 83.29, 71.86, 42.25, 54.26],
       [28.71, 71.33, 27.32, 83.29, 71.86, 42.25, 54.26],
       [28.71, 71.33, 27.32, 83.29, 71.86, 42.25, 54.26],
       [28.71, 71.33, 27.32, 83.29, 71.86, 42.25, 54.26],
       [28.71, 71.33, 27.32, 83.29, 71.86, 42.25, 54.26],
       [28.71, 71.33, 27.32, 83.29, 71.86, 42.25, 54.26],
       [28.71, 71.33, 27.32, 83.29, 71.86, 42.25, 54.26],
       [28.71, 71.33, 27.32, 83.29, 71.86, 42.25, 54.26],
       [28.71, 71.33, 27.32, 83.29, 71.86, 42.25, 54.26],
       [28.71, 71.33, 27.32, 83.29, 71.86, 42.25, 54.26],
       [28.71, 71.33, 27.32, 83.29, 71.86, 42.25, 54.26],
       [28.71, 71.33, 27.32, 83.29, 71.86, 42.25, 54.26],
       [28.71,

In [84]:
kernel = kernel - 0.5 * c - 0.5 * aa
kernel

array([[-7.365, -0.235, -6.49 , -1.325, -0.41 , -1.205, -6.13 ],
       [-0.87 , -7.53 , -1.225, -7.67 , -8.645, -3.99 , -0.465],
       [-3.185, -2.265, -2.33 , -3.585, -2.45 , -0.105, -4.12 ],
       [-8.985, -0.215, -8.05 , -0.205, -0.39 , -2.325, -6.07 ],
       [-0.77 , -6.9  , -1.355, -7.51 , -8.115, -3.75 , -0.385],
       [-8.21 , -0.14 , -7.145, -0.57 , -0.205, -1.53 , -6.275],
       [-3.315, -2.255, -2.37 , -3.465, -2.38 , -0.065, -4.25 ],
       [-1.635, -8.085, -2.3  , -7.885, -9.45 , -5.285, -0.15 ],
       [-0.335, -7.025, -0.7  , -7.965, -8.07 , -3.125, -0.93 ],
       [-5.215, -0.855, -4.31 , -2.065, -1.06 , -0.425, -4.95 ],
       [-7.065, -0.245, -6.15 , -0.755, -0.46 , -1.275, -5.25 ],
       [-0.03 , -8.55 , -0.255, -9.9  , -9.525, -3.42 , -1.865],
       [-0.69 , -6.87 , -1.185, -7.55 , -8.105, -3.63 , -0.425],
       [-6.85 , -0.52 , -5.755, -1.37 , -0.455, -0.73 , -6.165],
       [-0.06 , -8.23 , -0.195, -9.59 , -9.125, -3.11 , -1.995],
       [-0.375, -6.825, -

In [86]:
kernel = np.exp(kernel / (2. * 1.0 ** 2))
kernel

array([[0.02516   , 0.88914051, 0.03896856, 0.51556082, 0.81464732,
        0.54744132, 0.04665384],
       [0.64726467, 0.02316761, 0.54199419, 0.02160134, 0.01326668,
        0.13601365, 0.79254975],
       [0.20341643, 0.32222668, 0.31192266, 0.16654329, 0.2937577 ,
        0.94885432, 0.12745397],
       [0.01119263, 0.89807652, 0.01786342, 0.90257815, 0.82283466,
        0.31270344, 0.04807466],
       [0.68045064, 0.03174564, 0.50788512, 0.02340045, 0.0172922 ,
        0.15335497, 0.82489432],
       [0.01649002, 0.93239382, 0.02808555, 0.75201425, 0.90257815,
        0.46533393, 0.04339114],
       [0.19061492, 0.32384185, 0.30574618, 0.17684175, 0.30422126,
        0.96802245, 0.11943297],
       [0.44153411, 0.01755353, 0.31663677, 0.01939965, 0.00887071,
        0.07118309, 0.92774349],
       [0.84577662, 0.02982227, 0.70468809, 0.01863898, 0.01768568,
        0.20961139, 0.62813511],
       [0.07371861, 0.6521374 , 0.11590319, 0.35611556, 0.58860497,
        0.80856032, 0.0

The prediction and accuracy is usually higher than the linear kernel.

In [44]:
lambdas, support_vectors, support_vectors_id, b, targets, vector_number = train(train_data_set, train_labels, kernel_type='rbf')
predicted = classify_rbf(test_data_set, train_data_set, lambdas, targets, b, vector_number, support_vectors, support_vectors_id)
predicted = list(predicted.astype(int))

from sklearn.metrics import accuracy_score

print(accuracy_score(predicted, test_labels))

     pcost       dcost       gap    pres   dres
 0:  9.6305e+01 -1.2289e+03  2e+03  2e-01  2e-15
 1:  5.9143e+01 -1.2031e+02  2e+02  5e-03  2e-15
 2:  7.0898e+00 -1.6497e+01  2e+01  2e-16  2e-15
 3: -5.2057e-01 -3.7668e+00  3e+00  2e-16  1e-15
 4: -1.1712e+00 -1.8374e+00  7e-01  2e-16  4e-16
 5: -1.3952e+00 -1.6846e+00  3e-01  4e-16  3e-16
 6: -1.4671e+00 -1.5679e+00  1e-01  2e-16  3e-16
 7: -1.5060e+00 -1.5164e+00  1e-02  2e-16  3e-16
 8: -1.5105e+00 -1.5106e+00  1e-04  2e-16  3e-16
 9: -1.5105e+00 -1.5105e+00  1e-06  2e-16  4e-16
Optimal solution found.
0.85
